In [191]:
import pandas as pd

dft_data_path = '/Users/y1u0d2/desktop/Lab/data/stable-structure/'
nn_data_path = '/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization/10'

In [192]:
dft_lattice = pd.read_csv(f'{dft_data_path}/lattice_info.csv')
dft_rdf = pd.read_csv(f'{dft_data_path}/rdf.csv')
nn_lattice = pd.read_csv(f'{nn_data_path}/lattice.csv')
nn_rdf = pd.read_csv(f'{nn_data_path}/rdf.csv')
nn_lattice.rename(columns={'Cella':'a', 'Cellb':'b', 'Cellc':'c', 'CellAlpha':'alpha', 'CellBeta':'beta', 'CellGamma':'gamma'}, inplace=True)

In [173]:
def get_lattice_length_diff_df(index, dft_lattice, nn_lattice):
    nn_lattice_row = nn_lattice.iloc[index,:]
    structure = nn_lattice_row.structure
    idx = nn_lattice_row.idx
    dft_lattice_row_index = dft_lattice.reset_index().query(f'structure == "{nn_lattice_row.structure}"').index[0]
    dft_lattice_row = dft_lattice.iloc[dft_lattice_row_index,:]
    lattice_diff_sum = ((nn_lattice_row[2:5] - dft_lattice_row[1:4])**2).sum()
    diff_series = pd.Series([structure,idx,lattice_diff_sum],index=['structure', 'idx', 'lattice_length_diff'])
    return pd.DataFrame([diff_series])

lattice_diff_df = get_lattice_length_diff_df(0, dft_lattice, nn_lattice)
for i in range(len(nn_lattice)-1):
    tmp_diff_df = get_lattice_length_diff_df(i+1, dft_lattice, nn_lattice)
    lattice_diff_df = pd.concat([lattice_diff_df, tmp_diff_df])

In [174]:
def get_lattice_angle_diff_df(index, dft_lattice, nn_lattice):
    nn_lattice_row = nn_lattice.iloc[index,:]
    structure = nn_lattice_row.structure
    idx = nn_lattice_row.idx
    dft_lattice_row_index = dft_lattice.reset_index().query(f'structure == "{nn_lattice_row.structure}"').index[0]
    dft_lattice_row = dft_lattice.iloc[dft_lattice_row_index,:]
    lattice_diff_sum = ((nn_lattice_row[5:8] - dft_lattice_row[4:7])**2).sum()
    print(nn_lattice_row[5:8])
    diff_series = pd.Series([structure,idx,lattice_diff_sum],index=['structure', 'idx', 'lattice_angle_diff'])
    return pd.DataFrame([diff_series])

lattice_angle_diff_df = get_lattice_angle_diff_df(0, dft_lattice, nn_lattice)
for i in range(len(nn_lattice)-1):
    tmp_diff_df = get_lattice_angle_diff_df(i+1, dft_lattice, nn_lattice)
    lattice_angle_diff_df = pd.concat([lattice_angle_diff_df, tmp_diff_df])

alpha    90.0
beta     90.0
gamma    90.0
Name: 0, dtype: object
alpha    90.0
beta     90.0
gamma    90.0
Name: 1, dtype: object
alpha    90.0
beta     90.0
gamma    90.0
Name: 2, dtype: object
alpha    90.0
beta     90.0
gamma    90.0
Name: 3, dtype: object
alpha    90.0
beta     90.0
gamma    90.0
Name: 4, dtype: object
alpha    90.0
beta     90.0
gamma    90.0
Name: 5, dtype: object
alpha    90.0
beta     90.0
gamma    90.0
Name: 6, dtype: object
alpha    90.0
beta     90.0
gamma    90.0
Name: 7, dtype: object
alpha    90.0
beta     90.0
gamma    90.0
Name: 8, dtype: object
alpha    90.0
beta     90.0
gamma    90.0
Name: 9, dtype: object
alpha     90.0
beta      90.0
gamma    120.0
Name: 10, dtype: object
alpha         90.0
beta          90.0
gamma    120.00001
Name: 11, dtype: object
alpha     90.0
beta      90.0
gamma    120.0
Name: 12, dtype: object
alpha     90.0
beta      90.0
gamma    120.0
Name: 13, dtype: object
alpha         90.0
beta          90.0
gamma    119.99999
Name:

In [175]:
def get_rdf_diff_df(index, dft_rdf, nn_rdf):
    nn_rdf_row = nn_rdf.iloc[index,:]
    structure = nn_rdf_row.structure
    idx = nn_rdf_row.idx
    bond = nn_rdf_row.bond
    dft_rdf_row_index = dft_rdf.reset_index().query(f'structure == "{nn_rdf_row.structure}" and bond == "{bond}"').index[0]
    dft_rdf_row = dft_rdf.iloc[dft_rdf_row_index,:]
    diff_rdf = ((nn_rdf_row[3:] - dft_rdf_row[2:])**2).sum()
    diff_series = pd.Series([structure,idx,bond,diff_rdf],index=['structure', 'idx', 'bond', 'rdf_diff'])
    return pd.DataFrame([diff_series])

rdf_diff_df = get_rdf_diff_df(0, dft_rdf, nn_rdf)
for i in range(len(nn_rdf)-1):
    tmp_diff_df = get_rdf_diff_df(i+1, dft_rdf, nn_rdf)
    rdf_diff_df = pd.concat([rdf_diff_df, tmp_diff_df])
rdf_diff_df = rdf_diff_df.groupby(['structure', 'idx']).mean().reset_index()

In [176]:
df = pd.merge(lattice_diff_df, lattice_angle_diff_df, on=['structure', 'idx'])
df = pd.merge(df, rdf_diff_df, on=['structure', 'idx'])
df.to_csv(f'{nn_data_path}/diff.csv', index=False)

In [193]:
def _get_rdf_diff_df(index, dft_rdf, nn_rdf, another_structure=None):
    nn_rdf_row = nn_rdf.iloc[index,:]
    structure = nn_rdf_row.structure
    idx = nn_rdf_row.idx
    bond = nn_rdf_row.bond
    if another_structure is not None:
        dft_rdf_row_index = dft_rdf.reset_index().query(f'structure == "{another_structure}" and bond == "{bond}"').index[0]
    else:
        dft_rdf_row_index = dft_rdf.reset_index().query(f'structure == "{nn_rdf_row.structure}" and bond == "{bond}"').index[0]
    dft_rdf_row = dft_rdf.iloc[dft_rdf_row_index,:]
    diff_rdf = ((nn_rdf_row[3:] - dft_rdf_row[2:])**2).sum()
    diff_series = pd.Series([structure,idx,bond,diff_rdf],index=['structure', 'idx', 'bond', 'rdf_diff'])
    return pd.DataFrame([diff_series])

except_index1 = nn_rdf.query(f'structure == "hex-trydymite" and idx == 46').index[0]
except_index2 = nn_rdf.query(f'structure == "hex-trydymite" and idx == 46').index[1]
except_index3 = nn_rdf.query(f'structure == "hex-trydymite" and idx == 46').index[2]

df1 = _get_rdf_diff_df(except_index1, dft_rdf, nn_rdf, another_structure='beta-trydymite')
df2 = _get_rdf_diff_df(except_index2, dft_rdf, nn_rdf, another_structure='beta-trydymite')
df3 = _get_rdf_diff_df(except_index3, dft_rdf, nn_rdf, another_structure='beta-trydymite')

df = pd.concat([df1, df2, df3])

In [194]:
df.mean()

<ipython-input-194-c61f0c8f89b5>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.mean()


idx         46.00000
rdf_diff    36.69356
dtype: float64

In [195]:
def _get_diff_df(index, dft_lattice, nn_lattice, another_structure=None):
    nn_lattice_row = nn_lattice.iloc[index,:]
    structure = nn_lattice_row.structure
    idx = nn_lattice_row.idx
    if another_structure is not None:
        dft_lattice_row_index = dft_lattice.reset_index().query(f'structure == "{another_structure}"').index[0]
    else:
        dft_lattice_row_index = dft_lattice.reset_index().query(f'structure == "{nn_lattice_row.structure}"').index[0]
    dft_lattice_row = dft_lattice.iloc[dft_lattice_row_index,:]
    lattice_diff_sum = ((nn_lattice_row[2:5] - dft_lattice_row[1:4])**2).sum()
    diff_series = pd.Series([structure,idx,lattice_diff_sum],index=['structure', 'idx', 'lattice_diff'])
    return pd.DataFrame([diff_series])

except_index = nn_lattice.query(f'structure == "hex-trydymite" and idx == 46').index[0]
df = _get_diff_df(except_index, dft_lattice, nn_lattice, another_structure='beta-trydymite')

In [196]:
df

,structure,idx,lattice_diff
0,hex-trydymite,46,0.015681


In [197]:
def _get_angle_diff_df(index, dft_lattice, nn_lattice, another_structure=None):
    nn_lattice_row = nn_lattice.iloc[index,:]
    structure = nn_lattice_row.structure
    idx = nn_lattice_row.idx
    if another_structure is not None:
        dft_lattice_row_index = dft_lattice.reset_index().query(f'structure == "{another_structure}"').index[0]
    else:
        dft_lattice_row_index = dft_lattice.reset_index().query(f'structure == "{nn_lattice_row.structure}"').index[0]
    dft_lattice_row = dft_lattice.iloc[dft_lattice_row_index,:]
    lattice_diff_sum = ((nn_lattice_row[5:8] - dft_lattice_row[4:7])**2).sum()
    print(nn_lattice_row[5:8])
    print(dft_lattice_row[4:7])
    diff_series = pd.Series([structure,idx,lattice_diff_sum],index=['structure', 'idx', 'lattice_angle_diff'])
    return pd.DataFrame([diff_series])

except_index = nn_lattice.query(f'structure == "hex-trydymite" and idx == 46').index[0]
df = _get_angle_diff_df(except_index, dft_lattice, nn_lattice, another_structure='beta-trydymite')

alpha     90.0
beta      90.0
gamma    120.0
Name: 69, dtype: object
alpha      90.0
beta       90.0
gamma    121.77
Name: 5, dtype: object


In [198]:
df

,structure,idx,lattice_angle_diff
0,hex-trydymite,46,3.1329
